### CLICLICLIMATO ###

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  # Seaborn enhances the aesthetics of matplotlib plots
import pandas as pd
import xarray as xr
import random 

from tempest import casestudy
from tempest import grid
from tempest import storm_tracker
from tempest import handler
from tempest import joint_distrib

from tempest.plots.hist import simple_hist
settings_path = 'settings/sam3d.yaml'

In [6]:
# Instantiate CaseStudy by passing the settings. 
# Should also create appropriate directories
hdlr = handler.Handler(settings_path)
cs = casestudy.CaseStudy(hdlr, overwrite = False ,verbose = False)
st = storm_tracker.StormTracker(cs, overwrite_storms = False, overwrite = False, verbose = True) #overwrite = True is super long, computes growth rate (triangle fit)

Data loaded from /homedata/mcarenso/shear/SAM3d_Tropics/var_id_days_i_t.json
Loading storms...
loading storms from netcdf
Time elapsed for loading storms: 0.15 seconds


In [7]:
duration_min = 10
surfmaxkm2_min = 25000
save_storms_path = st.settings["DIR_DATA_OUT"]+f"save_storms_dmin{duration_min}_smin{surfmaxkm2_min}.nc"
if False : 
    storms = xr.open_dataset(st.file_storms)
    storms = storms.where(storms.INT_duration > 10, drop = True) # 1min
    storms = storms.where(storms.INT_surfmaxkm2_241K> 25000, drop = True)
    storms.to_netcdf(save_storms_path)
    storms.close
else : 
    storms = xr.open_dataset(save_storms_path)

## FileTracking is ft
ft = storms[[ 
    "INT_UTC_timeInit", "INT_UTC_timeEnd", "INT_duration", "INT_surfcumkm2_241K", "INT_velocityAvg", ## General characteristics
    "LC_lon", "LC_lat", "LC_UTC_time", "LC_ecc_241K", "LC_orientation_241K", "LC_surfkm2_241K", "LC_tb_90th", "LC_velocity" ## General characteristics
             ]]

test = xr.open_dataset("/bdd/DYAMOND/SAM-4km/OUT_3D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000001440_PP.nc")
targets = [300, 2000, 4000, 8000, 12000]
z_indices = [np.abs(test.z.values - target).argmin() for target in targets]
print(z_indices)
test.close()

FileNotFoundError: [Errno 2] No such file or directory: '/bdd/DYAMOND/SAM-4km/OUT_3D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000001440_PP.nc'

In [ ]:
## Valid times for 3d var are %6==0 
ft_3d = ft.sel(time=ft.time[(ft.time - st.i_t_start + cs.settings["TIME_RANGE"][0]) % 6 == 0])
iDCS = random.choice(ft_3d.DCS_number.values)

def get_frame_data(st, ft, iDCS):
    """
    returns everything to compute the frame and fields of a DCS over it's lifetime
    start and end corresponds to filetracking time index
    time_array corresponds to i_t of global variables (seg_mask, precip, U,...)
    """

    storm = ft.sel(DCS_number = iDCS)
    start = np.max([0,int(storm.INT_UTC_timeInit.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start])
    end = 1+int(storm.INT_UTC_timeEnd.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start
    lon_array, lat_array = storm.LC_lon[start:end].values, storm.LC_lat[start:end].values
    speed_array, time_array = storm.LC_velocity[start:end].values, storm.LC_UTC_time[start:end].values/1800 - st.i_t_start

    assert time_array[0]==start
    assert time_array[-1] +1 ==end

    time_array = time_array +st.settings["TIME_RANGE"][0]
    return start, end, lon_array, lat_array, speed_array, time_array


def get_valid_3d(cs):
    valid_3d = []
    for key in cs.days_i_t_per_var_id["U"].keys():
        for vt in cs.days_i_t_per_var_id["U"][key]:
            valid_3d.append(vt)
    return valid_3d

valid_3d = get_valid_3d(cs)


def get_frame_data_3d(st, ft, iDCS):
    """
    returns everything to compute the frame and fields of a DCS over it's lifetime
    start and end corresponds to filetracking time index
    time_array corresponds to i_t of global variables (seg_mask, precip, U,...)
    """

    storm = ft.sel(DCS_number = iDCS)
    start = np.max([0,int(storm.INT_UTC_timeInit.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start])
    end = 1+int(storm.INT_UTC_timeEnd.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start
    lon_array, lat_array = storm.LC_lon[start:end].values, storm.LC_lat[start:end].values
    speed_array, time_array = storm.LC_velocity[start:end].values, storm.LC_UTC_time[start:end].values/1800 - st.i_t_start

    # Initialize speed arrays with zeros
    speed_lon = np.zeros_like(lon_array)
    speed_lat = np.zeros_like(lat_array)
    for i in range(1, len(lon_array) - 1):
        speed_lon[i] = (lon_array[i + 1] - lon_array[i - 1]) / 2 #*delta_t
        speed_lat[i] = (lat_array[i + 1] - lat_array[i - 1]) / 2

    assert time_array[0]==start
    assert time_array[-1] +1 ==end

    time_array = time_array +st.settings["TIME_RANGE"][0]
    mask = np.isin(time_array, list(valid_3d))

    return start, end, lon_array[mask], lat_array[mask], speed_array[mask], time_array[mask], speed_lon[mask], speed_lat[mask]


def get_extent_slices(lons, lats, large_scale_frame_size=4):
    lon_min, lon_max = np.min(lons)-large_scale_frame_size, np.max(lons)+large_scale_frame_size
    lat_min, lat_max = np.min(lats)-large_scale_frame_size, np.max(lats)+large_scale_frame_size
    extent = [lon_min, lon_max, lat_min, lat_max]
    slice_lon = slice(lon_min, lon_max)
    slice_lat = slice(lat_min, lat_max)
    return extent, slice_lon, slice_lat


start, end, lons, lats, speeds, times = get_frame_data(st, ft, iDCS)
start, end, lons_3d, lats_3d, speeds_3d, times_3d, speed_lon_3d, speed_lat_3d = get_frame_data_3d(st, ft, iDCS)
extent, slice_lon, slice_lat = get_extent_slices(lons_3d, lats_3d)

print(times_3d)

In [ ]:
t = len(times_3d)


le_dico_long = {"latitude" : slice_lat, "longitude" : slice_lon}
le_dico_court = {    "lat" : slice_lat,       "lon" : slice_lon}

for i in range(t):
    # plot central cloud with contour TODO a func
    seg = hdlr.load_seg(times_3d[i])[0].sel(le_dico_long)
    nan_mask = np.isnan(seg)
    output = np.full_like(seg, 0.5, dtype = float)
    output[nan_mask] = 0
    output[seg == iDCS] = 1
    # contour = ax.contour(output, levels=[0.75], colors='red', linewidths=0.5, linestyles='solid', extent = extent, transform=ccrs.PlateCarree())
    output[seg == iDCS] = 0.5


    print(np.sqrt(speed_lon_3d[i]**2 + speed_lat_3d[i]**2))

    ### W ### 
    # list_w = [hdlr.load_var(cs, "W", times_3d[i], z_idx = z, sel_dict = le_dico_court) for z in z_indices]
    # for w in list_w:
    #     ax.imshow(w.W[0,0].values, origin = 'lower', cmap=w_cmap, interpolation='nearest', alpha = 0.3, norm=mcolors.Normalize(vmin=-1, vmax=1), extent = extent, transform=ccrs.PlateCarree()) #
    #     break

    ### U & V Hodograph ###
    list_u = [hdlr.load_var(cs, "U", times_3d[i], z_idx = z, sel_dict = le_dico_court) for z in z_indices]
    list_v = [hdlr.load_var(cs, "V", times_3d[i], z_idx = z, sel_dict = le_dico_court) for z in z_indices]
    hodo_colors = ["blue", "orange", "green", "red", "purple"]
    for u, v, La_Couleur_De_L_hodographe  in zip(list_u, list_v, hodo_colors):
        ## So what do we plot haha it depends on a radius ? square ? cloud_mask ? 
        u_clean, v_clean = u.U[0,0].values, v.V[0,0].values
        u_mean, v_mean = np.mean(u_clean), np.mean(v_clean)
        print(np.sqrt(u_mean**2 + v_mean**2))


    # plot prec (threshold, percentile based?)
    prec = hdlr.load_var(cs, "Prec", times_3d[i]).sel(le_dico_court).values
    prec_masked = np.ma.masked_less_equal(prec, 3)

## Finally show

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature


# Initialize figure and axis
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection': ccrs.PlateCarree()})

colors = [(1, 1, 1, 0), (0.8, 0.8, 0.8, 0.3), (0.5, 0.5, 0.5, 1)]  # RGBA for light grey to very light grey (with transparency)
cmap = LinearSegmentedColormap.from_list("light_grey_scale", colors)
prec_cmap =  plt.get_cmap("nipy_spectral")
w_cmap = plt.get_cmap("viridis")
prec_cmap.set_bad(color=(1, 1, 1, 0))  # I think that's useless but unsure the way maps are superimposed
cmap.set_bad(color=(1, 1, 1, 0))  # Transparent for NaNs
